<p style = "font-size : 50px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Customer Segmentation</strong></p>

<img style="margin-left: 10%; float: center;  border:5px solid #ffb037; width:80%; height : 80%;" src = https://profiletree.com/wp-content/uploads/2019/11/Segmentatie.jpg> 

In [ ]:
# importing libraries

## Import các thư viện (third-party) cần thiết cho việc thực hiện phân tích dữ liệu.
## Việc import này nhằm mục đích là có thể sử dụng các chức năng được tích hợp sẵn trong các thư viện.

# 'as xxx' ở đây là nhằm rút gọn code thay vì phải gọi dài dòng là pandas.xxxx ta chỉ cần gọi pd.xxx
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objs as go
import missingno as msno

## Import các module khác có sẵn trong project để sử dụng
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

import warnings 
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')  ## dùng để xác style cho notebook  'fivethirtyeight' là tham số có thể được thay đổi ví dụ 'ggplot'.Có thể tham khảo tại đây https://matplotlib.org/stable/tutorials/introductory/customizing.html
## chỉ đơn giản là để cái biểu đồ nằm trong khung hiển thị thôi.
%matplotlib inline 

In [ ]:
df = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv') #đọc dữ liệu từ file .csv (đường dẫn trong '' có thể thay đổi)
df.head() # lấy 5 dòng đầu của data được đọc từ file .csv. Có thể thay đổi số dòng = df.head(n) vs n là số nguyên dương.

In [ ]:
df.describe() # generate ra bảng thông tin static (count, %, ...)

In [ ]:
df.info() # in thông tin của các column trong file .csv ra màn hình console

In [ ]:
msno.bar(df) ## dùng để trực quan hóa sự phân bố các giá trị rỗng NaN trong data
plt.show() 

In [ ]:
# dropping useless column
df.drop('CustomerID', axis = 1, inplace = True) 

In [ ]:
# distribution plots

## Mục đích khối code này là tạo 1 biểu đồ có kích thước 20x8 trong đó có 3 biểu đồ nhỏ hơn theo từng tiêu chí
 
# vẽ các biểu đồ theo kích thước có thể thay đổi
plt.figure(figsize = (20, 8))

# khởi tạo só lượng biểu đồ
plotnumber = 1

# vẽ biểu đồ theo từng tiêu chí colum
## Chú ý: ['Age', 'Annual Income (k$)', 'Spending Score (1-100)'] các phần từ trong danh sách phải là column tồn tại trong data
for col in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
    if plotnumber <= 3:
        # thêm biểu đổ nhỏ trong biểu đồ to theo thứ tự 
        # Chú ý: bộ số subplot(1, 3, plotnumber) có thể thay đổi nhưng theo 1 số quy chuẩn khá là dài dòng.
        # Có thể hiểu đại khái là subplot(1, 3, plotnumber) chia cả biểu đồ to thành 3 biểu đồ dọc tỷ lệ 1/3 theo trục y
        # nếu để 2:2 thì là chia biểu đồ to thành 4 ô nhỏ eg: subplot(2,2, plotnumber)
        # nếu để 3:1 thì là chia biểu đồ to thành 3 biểu đồ ngang tỷ lệ 1:1 theo trục y
        ax = plt.subplot(1, 3, plotnumber)
        # vẽ biểu đồ đường lên trên biểu đồ nhỏ
        sns.distplot(df[col])
        
    plotnumber += 1

plt.tight_layout() #cho phép vẽ các mốc tọa độ chi tiết hơn (để 3:1 sẽ thấy sự thay đổi)
plt.show()

<ul>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>Age group near 30-40 have the highest density.</strong></li>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>Most customers have income in range of 50-80k.</strong></li>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>Most of the customers have spending score of 50.</strong></li>
</ul>

In [ ]:
values = df['Gender'].value_counts() # lấy số lượng của column 'Gender', có thể thay column khác
labels = ['Male', 'Female']

fig, ax = plt.subplots(figsize = (4, 4), dpi = 200) # đã giải thích ở phía trên thêm tham số dpi là độ phân giải mong muốn (có thể thay đổi)
explode = (0, 0.06)

## Vẽ biểu đồ tròn với các tham số có thể tham khảo ở đây https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html
# values là tổng các giá trị 
# labels là danh sách các giá trị trong column đấy có, có thể bỏ trống
# shadow: vẽ bóng hay ko False là ko vẽ True là có
# startangle: góc bắt đầu vẽ nếu ko đặt sẽ vẽ từ 0 độ
# explode: độ tịnh tiến của các phần (là khoảng cách giữa 2 mảnh)
patches, texts, autotexts = ax.pie(values, labels = labels, autopct = '%1.2f%%', shadow = True,
                                   startangle = 90, explode = explode)
# Chú ý: nếu thay 'Gender' = 1 column khác nên bỏ ko để tham số explode bởi lúc các cloumn khác nó có quá nhiều giá trị ko vẽ tịnh tiến tọa độ đc. labels cũng thế (labels là list các giá trị trong column đấy có)
# patches, texts, autotexts = ax.pie(values, autopct = '%1.2f%%', shadow = False, startangle = 90)

# đoạn dưới này thuần túy là setting màu vs cỡ chữ cho các phần có thể thêm các màu cỡ chữ khác nhau
plt.setp(texts, color = 'orange') 
plt.setp(autotexts, size = 12, color = 'white')
plt.setp(autotexts, size = 12, color = 'yellow')
autotexts[1].set_color('black')
plt.show()

In [ ]:
# Tương tự cái trên

plt.figure(figsize = (20, 8))
plotnumber = 1

for col in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
    if plotnumber <= 3:
        ax = plt.subplot(1, 3, plotnumber)
        # vẽ biểu đồ dạng gì đấy cũng ko rõ nữa
        sns.violinplot(x = col, y = 'Gender', data = df)
        
    plotnumber += 1
    
plt.tight_layout()
plt.show()

In [ ]:
# thực hiện lọc các giá trị tuổi theo khoảng và gán vào biến tương ứng
# có thể thêm ví dụ
tuoi_duoi_18 = df.Age[df.Age < 18]
age_18_25 = df.Age[(df.Age >= 18) & (df.Age <= 25)]
age_26_35 = df.Age[(df.Age >= 26) & (df.Age <= 35)]
age_36_45 = df.Age[(df.Age >= 36) & (df.Age <= 45)]
age_46_55 = df.Age[(df.Age >= 46) & (df.Age <= 55)]
age_55above = df.Age[df.Age >= 55]

In [ ]:
# nhãn legend
x_age = ['< 18', '18-25', '26-35', '36-45', '46-55', '55+']
# len(tuoi_duoi_18.values) số lượng các giá trị được lọc ra và gán từ bước trên
# trong đó tuoi_duoi_18.values sẽ list ra danh sách các giá trị được lưu vào tuoi_duoi_18
# len(tuoi_duoi_18.values) dếm tổng số giá trị đấy 
y_age = [len(tuoi_duoi_18.values), len(age_18_25.values), len(age_26_35.values), len(age_36_45.values), len(age_46_55.values),
     len(age_55above.values)]
## Chú ý số lượng phần từ ở x_age và y_age phải tương đương nhau 

# vẽ biểu đồ cột theo nhóm tuổi
px.bar(data_frame = df, x = x_age, y = y_age, color = x_age,
       title = 'Number of customers per age group')

In [ ]:
# là 1 dạng biểu đồ phân tán theo column x và y có thể đổi là column khác
px.scatter(data_frame = df, x = 'Annual Income (k$)', y = 'Spending Score (1-100)',
          title = 'Relation between Annual Income and Spending Score')

<ul>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>There is no relation between the customers having annual income in the range of 20-40k and 80-140k but there is relation between customers who have annual income in the range of 40-60k.</strong></li>
</ul>

In [ ]:
# tương tự trên 
ss_1_20 = df['Spending Score (1-100)'][(df['Spending Score (1-100)'] >= 1) &
                                       (df['Spending Score (1-100)'] <= 20)]

ss_21_40 = df['Spending Score (1-100)'][(df['Spending Score (1-100)'] >= 21) &
                                       (df['Spending Score (1-100)'] <= 40)]

ss_41_60 = df['Spending Score (1-100)'][(df['Spending Score (1-100)'] >= 41) &
                                       (df['Spending Score (1-100)'] <= 60)]

ss_61_80 = df['Spending Score (1-100)'][(df['Spending Score (1-100)'] >= 61) &
                                       (df['Spending Score (1-100)'] <= 80)]

ss_81_100 = df['Spending Score (1-100)'][(df['Spending Score (1-100)'] >= 81) &
                                       (df['Spending Score (1-100)'] <= 100)]

x_ss = ['1-20', '21-40', '41-60', '61-80', '81-100']
y_ss = [len(ss_1_20.values), len(ss_21_40.values), len(ss_41_60.values), len(ss_61_80.values),
        len(ss_81_100.values)]

px.bar(data_frame = df, x = x_ss, y = y_ss, color = x_ss,
       title = 'Number of customers per Spending Score group')

<ul>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>Most of the customers spending score is between 41-60k.</strong></li>
</ul>

In [ ]:
ai_0_30 = df['Annual Income (k$)'][(df['Annual Income (k$)'] >= 0) & (df['Annual Income (k$)'] <= 30)]
ai_31_60 = df['Annual Income (k$)'][(df['Annual Income (k$)'] >= 31)&(df['Annual Income (k$)'] <= 60)]
ai_61_90 = df['Annual Income (k$)'][(df['Annual Income (k$)'] >= 61)&(df['Annual Income (k$)'] <= 90)]
ai_91_120 = df['Annual Income (k$)'][(df['Annual Income (k$)']>= 91)&(df['Annual Income (k$)']<=120)]
ai_121_150 = df['Annual Income (k$)'][(df['Annual Income (k$)']>=121)&(df['Annual Income (k$)']<=150)]

x_ai = ['$ 0-30,000', '$ 30,001-60,000', '$ 60,001-90,000', '$ 90,001-120,000', '$ 120,000-150,000']
y_ai = [len(ai_0_30.values) , len(ai_31_60.values) , len(ai_61_90.values) , len(ai_91_120.values),
         len(ai_121_150.values)]

px.bar(data_frame = df, x = x_ai, y = y_ai, color = x_ai,
       title = 'Number of customers per Annual Income group')

<ul>
    <li style = "color : #03506f; font-size : 20px; font-family : 'Comic Sans MS';"><strong>Most of the customers annual income lies between 60k-90k.</strong></li>
</ul>

<p style = "font-size : 45px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : #f9b208; border-radius: 5px 5px;"><strong>K-Means</strong></p> 

<ul>
    <li style = "color : #f54748; font-size : 25px; font-family : 'Comic Sans MS';"><strong>kmeans clustering using `Age` and `Spending Score` columns</strong></li>
</ul>

In [ ]:
# df.loc cho phép lấy group column theo từng dòng xem ví dụ đây cho dễ hình dung https://www.w3resource.com/pandas/dataframe/dataframe-loc.php
X1 = df.loc[:, ['Age', 'Spending Score (1-100)']].values

# tạo tham số lưu giá trị của tính kmean
wcss= []
for k in range(1, 20):
    # function hỗ trợ tính KMeans trong Machine Learning (cái KMean này như nào a cũng ko biết e có tài liệu về AI hoặc ML này thì gửi a xem với)
    kmeans = KMeans(n_clusters = k, init = 'k-means++')
    kmeans.fit(X1)
    # add giá trị của tham số kmean vào
    wcss.append(kmeans.inertia_)

# tạo khung biểu đồ
plt.figure(figsize = (12, 7))

# vẽ đường 
## Chú ý range(1, 11) ở đây phải cùng giá trị vs range ở for k in range(1, 11)
## eg: for k in range(1, 20) -> plt.plot(range(1, 20), wcss, linewidth = 2, marker = '1')
## linewidth: đô rộng của đường line
## marker: marker sytle 
'''
'8'	Circle	 
'o'	Circle	
'*'	Star	
'.'	Point	
','	Pixel	
'x'	X	
'X'	X (filled)	
'+'	Plus	
'P'	Plus (filled)	
's'	Square	
'D'	Diamond	
'd'	Diamond (thin)	
'p'	Pentagon	
'H'	Hexagon	
'h'	Hexagon	
'v'	Triangle Down	
'^'	Triangle Up	
'<'	Triangle Left	
'>'	Triangle Right	
'1'	Tri Down	
'2'	Tri Up	
'3'	Tri Left	
'4'	Tri Right	
'|'	Vline	
'_'	Hline
'''
plt.plot(range(1, 20), wcss, linewidth = 2, marker = 'o')

# đặt tên cho biểu đồ 
plt.title('Elbow Plot\n', fontsize = 20)
# đặt tên 2 trục
plt.xlabel('K')
plt.ylabel('WCSS')
plt.show()

In [ ]:
#in ra kết quả gì đấy theo số lượng n_clusters = 1,2,...
kmeans = KMeans(n_clusters = 4)
labels = kmeans.fit_predict(X1)
print(labels)

In [ ]:
#in ra kết quả gì đấy
print(kmeans.cluster_centers_)

In [ ]:
# tương tự trên kia
plt.figure(figsize = (14, 8))

plt.scatter(X1[:, 0], X1[:, 1], c = kmeans.labels_, s = 105)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', s = 250)
plt.title('Clusters of Customers\n', fontsize = 20)
plt.xlabel('Age')
plt.ylabel('Spending Score (1-100)')
plt.show()

<ul>
    <li style = "color : #f54748; font-size : 25px; font-family : 'Comic Sans MS';"><strong>kmeans clustering using `Age` and `Annual Income` columns</strong></li>
</ul>

In [ ]:
# tương tự
X2 = df.loc[:, ['Age', 'Annual Income (k$)']].values

wcss= []
for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, init = 'k-means++')
    kmeans.fit(X2)
    wcss.append(kmeans.inertia_)

plt.figure(figsize = (12, 7))

plt.plot(range(1, 11), wcss, linewidth = 2, marker = '8')
plt.title('Elbow Plot\n', fontsize = 20)
plt.xlabel('K')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 5)
labels = kmeans.fit_predict(X2)
print(labels)

In [ ]:
print(kmeans.cluster_centers_)

In [ ]:
plt.figure(figsize = (12, 8))

plt.scatter(X2[:, 0], X2[:, 1], c = kmeans.labels_, s = 105)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', s = 250)
plt.title('Clusters of Customers\n', fontsize = 20)
plt.xlabel('Age')
plt.ylabel('Annual Income (k$)')
plt.show()

<ul>
    <li style = "color : #f54748; font-size : 25px; font-family : 'Comic Sans MS';"><strong>kmeans clustering using `Annual Income` and `Spending Score`</strong></li>
</ul>

In [ ]:
X3 = df.loc[:, ['Annual Income (k$)', 'Spending Score (1-100)']].values

wcss= []
for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, init = 'k-means++')
    kmeans.fit(X3)
    wcss.append(kmeans.inertia_)

plt.figure(figsize = (12, 7))

plt.plot(range(1, 11), wcss, linewidth = 2, marker = '8')
plt.title('Elbow Plot\n', fontsize = 20)
plt.xlabel('K')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 5)
labels = kmeans.fit_predict(X3)
print(labels)

In [ ]:
print(kmeans.cluster_centers_)

In [ ]:
plt.figure(figsize = (12, 8))

plt.scatter(X3[:, 0], X3[:, 1], c = kmeans.labels_, s = 105)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', s = 250)
plt.title('Clusters of Customers\n', fontsize = 20)
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.show()

<ul>
    <li style = "color : #f54748; font-size : 25px; font-family : 'Comic Sans MS';"><strong>kmeans clustering using `Age`, `Annual Score` and `Spending Score`</strong></li>
</ul>

In [ ]:
X4 = df.iloc[:, 1:]

wcss= []
for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, init = 'k-means++')
    kmeans.fit(X4)
    wcss.append(kmeans.inertia_)

plt.figure(figsize = (12, 7))

plt.plot(range(1, 11), wcss, linewidth = 2, marker = '8')
plt.title('Elbow Plot\n', fontsize = 20)
plt.xlabel('K')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 6)
clusters = kmeans.fit_predict(X4)
X4['label'] = clusters

In [ ]:
fig = px.scatter_3d(X4, x="Annual Income (k$)", y="Spending Score (1-100)", z="Age",
                    color = 'label', size = 'label')
fig.show()

<p style = "font-size : 45px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : #f9b208; border-radius: 5px 5px;"><strong>Hierarchical Clustering</strong></p> 

In [ ]:
plt.figure(figsize = (17, 8))

# vẽ dendogram từ dữ liệu X3 ở phía trên
dendo = dendrogram(linkage(X3, method = 'ward'))
plt.title('Dendrogram', fontsize = 15)
plt.show()

In [ ]:
agc = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage = 'ward')
labels = agc.fit_predict(X3)

In [ ]:
plt.figure(figsize = (12, 8))

plt.scatter(X3[labels == 0,0], X3[labels == 0,1], label = 'Cluster 1', s = 100)
plt.scatter(X3[labels == 1,0], X3[labels == 1,1], label = 'Cluster 2', s = 100)
plt.scatter(X3[labels == 2,0], X3[labels == 2,1], label = 'Cluster 3', s = 100)
plt.scatter(X3[labels == 3,0], X3[labels == 3,1], label = 'Cluster 4', s = 100)
plt.scatter(X3[labels == 4,0], X3[labels == 4,1], label = 'Cluster 5', s = 100)

plt.legend(loc = 'best')
plt.title('Clusters of Customers\n ', fontsize = 20)
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.show()

<p style = "font-size : 45px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : #f9b208; border-radius: 5px 5px;"><strong>DBSCAN</strong></p> 

In [ ]:
centers = [[1, 1], [-1, -1], [1, -1]] 
# để tạo data   https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4, 
                            random_state=0)  # generate sample blobs

X = StandardScaler().fit_transform(X)

In [ ]:
# DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)

# we are calculating these for showcasing in diagram
# creating an array of true and false as the same size as db.labels
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)  
core_samples_mask[db.core_sample_indices_] = True  # setting the indices of the core regions to True
labels = db.labels_  # similar to the model.fit() method, it gives the labels of the clustered data

In [ ]:
# Number of clusters in labels, ignoring noise if present.

# the label -1 is considered as noise by the DBSCAN algorithm
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) 
n_noise_ = list(labels).count(-1)  # calculating the number of clusters

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

"""Homogeneity metric of a cluster labeling given a ground truth.

A clustering result satisfies homogeneity if all of its clusters
contain only data points which are members of a single class."""

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))

In [ ]:
# Plot result
plt.figure(figsize = (10, 8))

# Black  is used for noise.
unique_labels = set(labels)  # identifying all the unique labels/clusters
colors = [plt.cm.Spectral(each)
          # creating the list of colours, generating the colourmap
          for each in np.linspace(0, 1, len(unique_labels))]  

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]
    class_member_mask = (labels == k)   # assigning class members for each class
    xy = X[class_member_mask & core_samples_mask] # creating the list of points for each class  
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),markeredgecolor='k', markersize=14)
    xy = X[class_member_mask & ~core_samples_mask] # creating the list of noise points   
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14)
    
plt.title('Clustering using DBSCAN\n', fontsize = 15)
plt.show()

<p style = "color : #f54748; font-size : 35px; font-family : 'Comic Sans MS';"><strong>If you like this kernel, please do upvote</strong></p>
